In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import numpy as np
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import math

## data

In [ ]:
#data from Book Crossing, filled in with web scrapping and then cleaned
books = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_cleaned_w2v.csv"

#ratings from Book Crossing
ratings = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings.csv"

#Created ratings (from the BookCrossing ones), in a trial to balance classes (see previous 'Balancing classes')
ratings_bin = "/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv"

In [ ]:
#Cleaned data base with scrapped books
df_books = pd.read_csv(books, sep = "|")

In [ ]:
#ratings of Book Crossing
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

In [ ]:
print(len(df_ratings))
df_ratings.head()

1149780


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
df_books.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ratings[df_ratings.ISBN == "000648302X"]

,User-ID,ISBN,Book-Rating
45384,11676,000648302X,8
174443,37950,000648302X,9
373708,89624,000648302X,9
412327,98544,000648302X,0
417691,99328,000648302X,3
496961,119654,000648302X,8
499864,120610,000648302X,7
654880,158589,000648302X,8
659263,160016,000648302X,4
718973,174092,000648302X,0


# Bert on books descriptions (already trained)

### Loading

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 7.4MB/s 
     |████████████████████████████████| 3.3MB 24.7MB/s 
     |████████████████████████████████| 901kB 41.3MB/s 


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
#I will use an already trained BERT network
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Tokenizer and embedding

In [ ]:
#Use of pretrained model
#Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
#Use of GPU to save time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
#copy of the books' database
df_bert = df_books.copy()

In [ ]:
#I only deal with non empty "book_description" cells

lineOK = df_bert[(df_bert["book_description"].isna() == False) & (df_bert["book_language"] == "en")]
len(lineOK)

64229

In [ ]:
#Creation of an additional column in order to put the future book_description embedding
df_bert = df_bert.assign(description_bert = 0)

In [ ]:
#execution in about 16min

#from transformers import BertTokenizer
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#model = BertModel.from_pretrained('bert-base-uncased'...
#forward(input_ids=None, attention_mask=None, ...

with torch.no_grad():
  for i_line in lineOK.index:
    
    #one book description, potentially composed of several sentences
    sent_list = df_bert.book_description[i_line].split(".")
    #I only keep the not empty results of the previous split
    sent_list = [sent for sent in sent_list if len(sent) > 3]

    #Output of sentences tokenization
    input_ids = []
    attention_masks = []

    #For each english sentence in the current book_description (not empty)
    ######################
    # Tokenization
    ######################
    for sent in sent_list:

      #print(len(sent.split()))
      #Tokenize all of the sentences and map the tokens to thier word IDs.
      encoded_dict = tokenizer.encode_plus(
                                  sent,                      # Sentence to encode.
                                  add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                  max_length = 64,           # Pad & truncate all sentences.
                                  pad_to_max_length = True,
                                  return_attention_mask = True,   # Construct attn. masks.
                                  return_tensors = 'pt',     # Return pytorch tensors.
                        )
              
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
              
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

    ######################
    # Embeddings
    ######################
    if (len(input_ids) > 0):
      # Convert the lists into tensors.
      input_ids_t = torch.cat(input_ids, dim = 0)
      attention_masks_t = torch.cat(attention_masks, dim = 0)

      #use Bert to embedds each tokenized sentences of the current book_description
      outputs = model(input_ids_t.to(device), attention_masks_t.to(device))
      hidden_states = outputs[2]

      # `hidden_states`       has shape [13 layers, len(input_ids), 64, 768] 
      # => hidden_states[-2]  has shape [len(input_ids), 64, 768] 
      token_vecs = hidden_states[-2]
      #print(token_vecs.shape)

      #mean elements by elements of each token (64 tokens in total) of each embedded sentences of the current book_description
      #=> sentence_embedding  has shape [len(input_ids), 768]: 
      sentence_embedding = torch.mean(token_vecs, dim=1)
      #print(sentence_embedding.shape)

      #I only want to keep one embedding of dim 768 for each book_description
      #(and not len(input_ids) x 768)
      sentence_embedding = torch.mean(sentence_embedding, dim=0)
      #print(sentence_embedding.size())


      ######################
      # Saving
      ######################
      #I get the values from GPU, and I put it in df_bert.description_bert column
      sentence_embedding = sentence_embedding.cpu().numpy()
      l = len(sentence_embedding)
      df_bert.description_bert[i_line] = [str(sentence_embedding[i]) for i in range(l)]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_bert.book_description[lineOK.index[0]])
print('Token IDs:', df_bert.description_bert[lineOK.index[0]])

Original:  With 160 million copies of the Twilight Saga sold worldwide, this addictive love story between a teenage girl and a vampire redefined romance for a generation. Here&#39;s the book that started it all.
Token IDs: ['-0.098250076', '-0.38007897', '0.8143984', '-0.069222644', '0.16267884', '-0.36887923', '0.09200936', '0.4111976', '0.32065162', '-0.0675951', '-0.17918313', '-0.24016543', '0.19495136', '0.4354491', '0.41287875', '0.4453775', '-0.24839687', '0.16070387', '0.14195046', '-0.16554643', '0.30131242', '0.28766793', '-0.12406763', '0.17628828', '0.21566428', '0.37729064', '-0.3119564', '0.008895561', '-0.51559365', '0.3141598', '0.25739694', '-0.24879043', '0.24140932', '0.0896724', '-0.38097662', '-0.012144949', '-0.15168047', '0.021698063', '-0.23981865', '0.23302785', '-0.5429584', '-0.34142113', '0.39717984', '-0.47054458', '-0.12300413', '-0.014472974', '0.20256266', '0.048919663', '-0.06945287', '-0.14903395', '-0.48877', '-0.08599328', '0.078301966', '-0.1906274'

In [ ]:
df_bert.to_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|", index = False)

# Fast Text on books descriptions


### Sentences tokenized

In [ ]:
import re
import nltk

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

from nltk import WordPunctTokenizer
word_punctuation_tokenizer = nltk.WordPunctTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Before embedding, book's description sentences need to be cleaned

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [ps.stem(word) for word in tokens]
        tokens = [word for word in tokens if word not in stopwords_list]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

In [ ]:
#I only use the not empty book_description lines, and english books
lineOK = df_books[(df_books["book_description"].isna() == False) & (df_books["book_language"] == "en")]
len(lineOK)

64229

In [ ]:
print(len(lineOK))
df_books.book_description[lineOK.index[0]]

64229


'With 160 million copies of the Twilight Saga sold worldwide, this addictive love story between a teenage girl and a vampire redefined romance for a generation. Here&#39;s the book that started it all.'

In [ ]:
df_books.book_description[lineOK.index[1]]

'Bestselling author John Green returns with an eagerly anticipated and emotional novel about sickness and health, life and death.'

In [ ]:
#list of all sentences
final_corpus = []

for i_line in lineOK.index:
    
  #one book description, potentially composed of several sentences
  sent_list = df_books.book_description[i_line].split(".")

  #for each sub-sentences of the description: tokenization
  cleaned_sent = [preprocess_text(sent) for sent in sent_list if sent.strip() !='']

  #for each tokenized sub-sentences, a last cleaning
  word_tokenized_corpus = []
  for sent in cleaned_sent:
    word_tokenized_corpus.extend(word_punctuation_tokenizer.tokenize(sent))

  #only one list for all sub-sentences of the same book's description
  final_corpus.append(word_tokenized_corpus)


In [ ]:
cleaned_sent

['bestsel author john green return eagerli anticip emot novel sick health life death']

In [ ]:
final_corpus

[['million',
  'copi',
  'twilight',
  'saga',
  'sold',
  'worldwid',
  'addict',
  'love',
  'stori',
  'teenag',
  'girl',
  'vampir',
  'redefin',
  'romanc',
  'gener',
  'book',
  'start'],
 ['bestsel',
  'author',
  'john',
  'green',
  'return',
  'eagerli',
  'anticip',
  'emot',
  'novel',
  'sick',
  'health',
  'life',
  'death']]

In [ ]:
len(final_corpus)

64229

### Training

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 40
min_word = 1
down_sampling = 1e-2

In [ ]:
#It takes about 1H 10min for about 65 000 lines

#window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that I want to create skip-gram model.

ft_model = FastText(final_corpus,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
ft_model.save("/content/drive/MyDrive/ProjetML/FastText_desc.model")

### Save of embeddings

In [ ]:
ft_model = FastText.load("/content/drive/MyDrive/ProjetML/FastText_desc.model")

In [ ]:
df_fastFM = df_books.copy()

#Creation of an additional column in order to put the future book_description embedding
df_fastFM = df_fastFM.assign(description_fastFM = 0)

#I only use the not empty book_description lines, and english books (same selection as previously for training)
lineOK = df_books[(df_books["book_description"].isna() == False) & (df_books["book_language"] == "en")]
len(lineOK)

64229

In [ ]:
df_fastFM.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards,description_fastFM
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
for i in range(len(lineOK.index)):

  #a tokenized sentences
  sent = final_corpus[i]

  if (len(sent) > 0):
    #find the embedding (vector of 60) for all words inside tokenized sentence
    words_emb = [ft_model.wv[w] for w in sent]

    emb = [0.0]*60

    #Perform the sum of each words embeddings
    for word in words_emb:
      l = len(word)
      #sum of embedding of each words
      emb = [emb[k] + word[k] for k in range(l)]

    emb = [emb[k] / len(words_emb) for k in range(l)]

    # Saving
    df_fastFM.description_fastFM[lineOK.index[i]] = [str(emb[i]) for i in range(l)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', df_fastFM.book_description[lineOK.index[0]])
print('Token IDs:', df_fastFM.description_fastFM[lineOK.index[0]])

Original:  With 160 million copies of the Twilight Saga sold worldwide, this addictive love story between a teenage girl and a vampire redefined romance for a generation. Here&#39;s the book that started it all.
Token IDs: ['0.046032028402859235', '0.15718470524777384', '-0.1395825319649542', '-0.1819172301055754', '0.17207935299066937', '-0.1365122384775211', '0.16689892704872525', '0.33930738577071357', '-0.06989275658612742', '0.31512450802085157', '-0.10388041380792856', '0.022007644121699473', '1.6696477199302e-05', '0.3797191175467828', '-0.273707770742476', '-0.20886098593473434', '0.11984747141490087', '-0.10271839490708183', '-0.09797142072142485', '-0.23660339820472634', '0.2803903391913456', '0.11936031935757081', '0.024216600488323498', '0.0775216789368321', '-0.014470523113713545', '-0.09681690356913297', '-0.09977425330811564', '0.1142483646159663', '-0.2905912635957493', '0.026502975954345483', '0.2798446424524574', '-0.09337197529042468', '0.21702254563570023', '-0.1523

In [ ]:
df_fastFM.to_csv("/content/drive/MyDrive/ProjetML/Books_description_fastText.csv", sep = "|", index = False)

# Fast Text on AuthorContext
- book_title 
- book_author 
- Category 
- Category_other 
- author_genres 
- author_genres_other

### Information preparation

In [ ]:
#I keep only few columns to create the context of training

#I only use english books
df1 = df_books[df_books["book_language"] == "en"]
df1 = df1[["book_title", "book_author", "Category", "Category_other", "author_genres", "author_genres_other"]]
print(len(df_books))
print(len(df1))
df1[1000:1005]

105508
93808


,book_title,book_author,Category,Category_other,author_genres,author_genres_other
1010,Shadowland : The Immortals,Alyson Noël,young adult fict,NaN,young adult,NaN
1011,You Can Heal Your Life,Louise L. Hay,chemistri,psycholog,religion,spiritu
1013,I Am Legend,Richard Matheson,horror tal,NaN,science fict,fantasi
1014,Rules of Prey,John Sandford,fiction,NaN,mysteri,thriller
1015,The Yiddish Policemen's Union : A Novel,Michael Chabon,fiction,NaN,literatur,fiction


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#with data set of about 100 000 lines, it takes 1 min

#I only deal with not empty book_title cells
lines_OK = df1.book_title[df1.book_title.isna() == False]

#Cleaning of the book title cells
for i in lines_OK.index:
    #I get sentence inside book_title column, I remove final '.' and separate all words
    w = df1.book_title[i].strip('.').split()

    #I remove stopwords, words of length < 2 and '...'
    ww = [word.strip(',') for word in w if (word not in stopwords_list) and len(word) > 2 and (word != '...')]

    #I reconstruct a sentence with the remaining words, separated with ','
    df1.book_title[i] = ','.join(ww)

In [ ]:
print(len(df1))
df1.book_title.head()

93808


0    The,Hunger,Games,The,First,Book,Hunger,Games
1                                        Twilight
2                             The,Fault,Our,Stars
3                                   Angels,Demons
4                                       Divergent
Name: book_title, dtype: object

In [ ]:
#All those columns informations must be joined inside one column
#str(x) != "nan":     but I join only if column word is not empty
#x.split():           sentences in colum 'book_title' are cut in words
#','.join(x.split()): all those words are joined with (',') to be transformed after in list of list
def WordToJoin(w):
  ww = [x for x in w if str(x) != "nan"]
  return ','.join(ww)

df2 = df1.apply(lambda x: WordToJoin(x), axis = 1)
df2.head()

0    The,Hunger,Games,The,First,Book,Hunger,Games,S...
1    Twilight,Stephenie Meyer,young adult fict,scie...
2    The,Fault,Our,Stars,John Green,juvenile fict,l...
3     Angels,Demons,Dan Brown,fiction,mysteri,thriller
4    Divergent,Veronica Roth,juvenile fict,science ...
dtype: object

In [ ]:
#a specific data frame is created with all this train context
df_clean = pd.DataFrame({'clean': df2})
df_clean.head()

,clean
0,"The,Hunger,Games,The,First,Book,Hunger,Games,S..."
1,"Twilight,Stephenie Meyer,young adult fict,scie..."
2,"The,Fault,Our,Stars,John Green,juvenile fict,l..."
3,"Angels,Demons,Dan Brown,fiction,mysteri,thriller"
4,"Divergent,Veronica Roth,juvenile fict,science ..."


In [ ]:
df_clean.iloc[11]["clean"]

'Lord,Flies,William Golding,fiction,literatur,fiction'

In [ ]:
#list of lists creation: 1 list for each book
sent = [row.split(',') for row in df_clean["clean"]]

print(len(sent))
sent[0:5]

93808


[['The',
  'Hunger',
  'Games',
  'The',
  'First',
  'Book',
  'Hunger',
  'Games',
  'Suzanne Collins',
  'juvenile fict'],
 ['Twilight',
  'Stephenie Meyer',
  'young adult fict',
  'science fict',
  'fantasi'],
 ['The',
  'Fault',
  'Our',
  'Stars',
  'John Green',
  'juvenile fict',
  'literatur',
  'fiction'],
 ['Angels', 'Demons', 'Dan Brown', 'fiction', 'mysteri', 'thriller'],
 ['Divergent', 'Veronica Roth', 'juvenile fict', 'science fict']]

### Training

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 40
min_word = 1
down_sampling = 1e-2

In [ ]:
#The training has been performed without Colab (problem of session crach)
#------------------------------------------------------------------------

#window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned 

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that I want to create skip-gram model.

ft_model_auth = FastText(sent,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
#ft_model_auth.save("/content/drive/MyDrive/ProjetML/FastText_auth.model")

### Save of embeddings

In [ ]:
ft_model_auth = FastText.load("/content/drive/MyDrive/ProjetML/FastText_auth_bis.model")

In [ ]:
df_fastText_auth = df_books.copy()
print("len(df_fastText_auth)", len(df_fastText_auth))

#I only embeds title of english books
df_fastText_auth = df_fastText_auth[df_fastText_auth["book_language"] == "en"]
print("len(df1)", len(df_fastText_auth))

#I only embeds not empty title (same selection as previously for training)
lines_OK = df_fastText_auth[df_fastText_auth.book_title.isna() == False]
print("len(lines_OK)", len(lines_OK), '\n')
#must be 93808

len(df_fastText_auth) 105508
len(df1) 93808
len(lines_OK) 93808 



In [ ]:
#It takes about 30min

#Creation of 60 columns for title embeddings
col = ["c_" + str(i) for i in range(60)]
#I will save ISBN and title embeddings in a new dataframe
df = pd.DataFrame(columns = ["isbn"] + col)

for i in range(len(lines_OK.index)):

  #a tokenized sentences
  sent_auth = sent[i]
  #print("len(sent)", len(sent))

  if (len(sent_auth) > 0):
    #find the embedding (vector of 60) for all words inside tokenized sentence
    words_emb = [ft_model_auth.wv[w] for w in sent_auth]

    emb = [0.0]*60

    #Perform the sum of each words embeddings
    for word in words_emb:
      l = len(word)
      #sum of embedding of each words
      emb = [emb[k] + word[k] for k in range(l)]

    emb = [emb[k] / len(words_emb) for k in range(l)]

    # Saving
    df.loc[i] = [str(df_fastText_auth.isbn[lines_OK.index[i]])] + emb

In [ ]:
df.head()

,isbn,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,0439023483,-1.133914,0.287926,-0.340693,0.304958,0.298398,-0.173713,-0.143411,0.229791,0.161779,-0.452773,0.651183,-0.976731,-0.251009,0.249334,-0.425694,-0.049007,-0.170870,0.213249,0.627869,-0.062812,0.058515,-0.288496,0.009767,-0.097794,0.179403,-0.282955,0.197258,0.012673,0.048874,-0.422243,0.032915,-0.418987,-0.094094,-1.181253,0.203269,-0.195160,0.066798,0.305457,-0.251446,-0.330791,0.604022,-0.311264,0.439883,0.393421,-0.087169,-0.278171,-0.129530,0.384977,-0.419687,0.456741,0.174104,0.716973,-0.182069,0.037079,-0.085198,-0.071911,0.257613,0.467097,0.058294,0.152557
1,0316015849,-0.661967,0.020827,-0.102629,0.656167,-0.522500,-0.686821,-0.081805,-0.155748,0.310289,-1.080961,0.280845,-1.200535,-1.021454,0.434017,-0.708987,0.037595,-0.258162,0.517609,0.635528,-0.027594,0.039133,-0.095607,0.306347,0.124747,0.379338,0.349878,-0.797516,0.003671,0.246132,-0.511566,-0.860476,-1.117019,-0.060744,-0.885507,0.515735,-0.011868,-0.119200,-0.016419,-0.142642,-0.626561,0.135914,-0.473242,0.192619,0.394811,0.276300,-0.479259,0.379815,0.882635,-0.855744,-0.262017,0.313448,0.648078,0.309740,-0.321710,0.097637,-0.480251,-0.052613,0.416854,0.032663,0.339096
2,0525478817,-0.637566,0.209719,0.069093,0.683498,0.282869,-0.387616,0.082728,-0.001888,0.177101,-0.487846,0.046389,-0.829359,-0.239804,0.080183,-0.277901,0.169195,-0.670780,0.463394,0.423762,-0.241552,0.088850,0.185821,-0.285995,-0.024849,0.241534,0.173943,-0.224615,0.195107,0.282236,-0.269725,0.167377,-0.690227,0.043004,-0.564121,0.064312,-0.254965,0.369041,0.389571,-0.097877,-0.234326,0.499018,-0.692860,0.178148,0.335409,-0.208531,-0.750370,-0.239607,0.228991,-0.475618,0.431623,0.124231,0.234835,0.406248,-0.005727,-0.268372,-0.230530,0.271181,0.400645,-0.258210,0.140380
3,1416524797,-0.909146,0.381753,-0.004445,0.266136,-0.924229,-0.401858,-0.179810,0.444664,0.405656,-0.297708,0.376238,-0.658277,0.473716,0.480343,-0.410677,0.068508,-0.700602,0.684635,0.219251,-0.247191,-0.272314,0.008100,-0.668079,0.581460,0.519319,0.248770,-0.332152,0.404158,0.631275,-0.592349,0.208009,-0.510605,-0.044259,-0.373369,0.451790,-0.497317,-0.225118,0.213629,-0.261501,-0.345028,0.516648,-0.574433,0.115345,0.881785,-0.527098,-0.375186,0.099308,0.707286,-0.424103,0.045841,0.179221,0.679570,0.355954,-0.060177,-0.182488,-0.192093,-0.096261,0.018247,-0.245425,0.129010
4,0062024035,-1.071675,0.272695,-0.132930,0.528883,-0.212030,-0.668376,-0.155125,-0.208077,0.675720,-1.170474,0.376464,-1.580428,-0.810530,0.406683,-0.440038,-0.073778,-0.330848,0.372500,0.224912,-0.433820,-0.104965,0.294745,0.282019,-0.010944,-0.053917,0.576428,-1.087371,-0.090118,-0.001131,-0.061287,-0.709023,-0.981501,-0.116600,-0.745491,0.449229,-0.345509,0.150247,0.121468,-0.246103,-0.303201,0.458214,-0.775023,0.454326,0.292916,0.352331,-0.115261,0.562451,0.256101,-0.935672,0.312968,0.071828,0.650803,-0.066576,-0.794026,0.215217,-0.469530,0.362375,0.602193,0.033010,0.206395


In [ ]:
df.to_csv("/content/drive/MyDrive/ProjetML/authorsContext_bis_fastText.csv", sep = "|", index = False)

# FastText on title

### Information preparation

In [ ]:
#I only use the not empty title lines, and english books
df1 = df_books[df_books["book_language"] == "en"].copy()
lines_OK = df1[df1.book_title.isna() == False]

df1 = df1["book_title"]
print("len(df_books)", len(df_books))
print("len(df1)", len(df1))
print("len(lines_OK)", len(lines_OK), '\n')
df1[1000:1005]

len(df_books) 105508
len(df1) 93808
len(lines_OK) 93808 



1010                 Shadowland : The Immortals
1011                     You Can Heal Your Life
1013                                I Am Legend
1014                              Rules of Prey
1015    The Yiddish Policemen's Union : A Novel
Name: book_title, dtype: object

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sent = []

for i in lines_OK.index:
    #I get sentence inside book_title column, I remove final '.' and separate all words
    w = df1[i].strip('.').split()
    #print(w)

    #I remove stopwords, words of length < 2 and '...'
    ww = [word.strip(',') for word in w if (word not in stopwords_list) and len(word) > 2 and (word != '...')]
    #print(ww)

    #I reconstruct a sentence with the remaining words, separated with ','
    sent = sent + [ww]
    #print(sent)

In [ ]:
#list of lists creation: 1 list for each book
print(len(sent))
sent[15000:15005]

93808


[['Hurricane', 'Bay'],
 ['Dumped', 'Anthology'],
 ['10th', 'Grade', 'Novel'],
 ['Two-in-One', 'Crosswords', 'Hard', 'Easy', 'Clues', 'Every', 'Puzzle'],
 ['Rock', "Picker's", 'Guide', 'Lake', "Superior's", 'North', 'Shore']]

### Training

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 10
min_word = 1
down_sampling = 1e-2

In [ ]:
#It takes about almost 10min

#window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that I want to create skip-gram model.

ft_model_title = FastText(sent,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
ft_model_title.save("/content/drive/MyDrive/ProjetML/FastText_title.model")

### Save embeddings

In [ ]:
df_books.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",young adult fict,NaN,With 160 million copies of the Twilight Saga s...,en,NaN,544.0,science fict,fantasi,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,juvenile fict,NaN,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318.0,literatur,fiction,[],4.19,West Australian Young Readers' Book Award
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,fiction,NaN,This is the book that started it all: we meet ...,en,NaN,736.0,mysteri,thriller,"['https://www.goodreads.com/book/show/960', 'h...",3.91,NaN
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,juvenile fict,NaN,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576.0,science fict,NaN,['https://www.goodreads.com/book/show/18080920...,4.18,Lincoln Award Nominee


In [ ]:
df_fastText_title = df_books.copy()
print("len(df_fastText_title)", len(df_fastText_title))

#I only embeds title of english books
df_fastText_title = df_fastText_title[df_fastText_title["book_language"] == "en"]
print("len(df1)", len(df_fastText_title))

##I only embeds not empty title (same selection as previously for training)
lines_OK = df_fastText_title[df_fastText_title.book_title.isna() == False]
print("len(lines_OK)", len(lines_OK), '\n')

len(df_fastText_title) 105508
len(df1) 93808
len(lines_OK) 93808 



In [ ]:
#It takes about 30min

#Creation of 60 columns for title embeddings
col = ["c_" + str(i) for i in range(60)]
#I will save ISBN and title embeddings in a new dataframe
df = pd.DataFrame(columns = ["isbn"] + col)

for i in range(len(lines_OK.index)):

  #a tokenized sentences
  sent_title = sent[i]
  #print(sent_title)

  if (len(sent_title) > 0):
        
    #find the embedding (vector of 60) for all words inside tokenized sentence
    words_emb = [ft_model_title.wv[w] for w in sent_title]

    emb = [0.0]*60

    #Perform the sum of each words embeddings
    for word in words_emb:
      l = len(word)
      #sum of embedding of each words
      emb = [emb[k] + word[k] for k in range(l)]

    emb = [emb[k] / len(words_emb) for k in range(l)]

    # Saving
    df.loc[i] = [str(df_fastText_title.isbn[lines_OK.index[i]])] + emb


In [ ]:
print(words_emb)
print(len(words_emb))
print(len(df))
df.head()

[array([ 0.07796233,  0.58427906, -0.52977955,  0.58431965,  0.18091802,
       -0.8304771 , -0.3028831 , -0.05778762, -0.81009585, -0.00312023,
       -0.14222343,  0.63524354, -0.67769396,  0.82649666,  0.7919355 ,
        0.68809754, -0.7118478 , -0.19658211, -0.63528967, -0.33240667,
       -0.38457733, -0.23635124, -0.59232897,  0.13039663, -0.877353  ,
       -1.0088793 ,  0.11770169,  0.2757143 , -0.17188446,  0.07871471,
        0.03902444,  0.10720091, -0.40888244, -1.2803923 ,  1.0502542 ,
       -0.11735027,  0.06935487, -0.17090222,  0.48304766,  0.38350457,
       -0.66694146,  0.63072   ,  0.11684398,  0.22092728, -0.1672962 ,
        0.709976  ,  0.6582761 , -0.05447186, -0.2570834 ,  0.06008875,
       -0.8126718 ,  0.6706839 ,  0.64300674, -0.68159056, -0.08373591,
       -0.15182897,  0.74320364, -0.5717664 , -0.20092306, -0.01392402],
      dtype=float32), array([-0.30528572,  1.0905325 , -0.91174084,  0.6413695 ,  0.29786843,
       -0.93775904, -0.708566  , -0.6121

,isbn,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,0439023483,-0.005422,0.660218,-0.328451,0.196469,-0.208463,-0.739556,0.003857,0.000029,-0.108310,-0.188837,0.121907,0.466777,-0.005632,0.108326,0.507619,-0.347326,-0.440195,0.084967,0.028980,0.452554,0.684329,0.005698,-0.028753,0.099497,-0.118867,-0.642309,-0.550026,-0.003231,-0.198943,-0.000068,0.006332,0.712410,-0.290316,-0.390195,0.938969,-0.497423,0.084140,0.802105,0.138267,-0.527298,0.215962,-0.016273,-0.427832,-0.078170,-0.662567,-0.270726,0.162164,0.049534,-0.254948,0.155271,-1.389042,0.280754,0.239190,-0.649777,-0.455659,-0.250190,0.029539,-0.688481,-0.115078,-0.199327
1,0316015849,-0.776157,1.205882,-0.359419,0.618401,-0.793910,0.095765,-0.028470,-0.032119,-0.031577,0.505846,-0.213235,1.077281,0.343695,1.067593,0.147618,0.346520,0.332632,-0.441578,-0.205350,-0.247127,-0.302588,-0.225925,0.139530,-0.513730,0.290471,-0.298226,-1.227239,-0.352734,-0.904764,0.498051,-0.579458,0.861378,0.071180,-0.609695,0.636024,0.346209,0.365903,-0.320421,-0.375289,-0.695907,1.323531,0.120134,-0.209622,-0.675126,-0.676011,-0.086022,1.285815,0.027476,0.522839,1.502978,-0.887846,0.490602,-0.731530,-0.512815,-0.771496,-0.323455,-0.345502,-0.840311,0.608957,0.540533
2,0525478817,-0.520020,1.009890,-0.616962,0.606713,-0.603252,-0.378865,0.026071,-0.006697,-0.491275,-0.178487,-0.004465,0.184280,-0.216923,0.098955,-0.243836,-0.164421,-0.107281,-0.426754,-0.248151,-0.191235,0.501240,-0.693983,-0.519364,-0.469612,0.053431,-0.601493,-0.372704,0.166216,-0.054395,-0.008349,-0.274709,0.378998,-0.202416,-0.408071,0.445716,-0.097853,0.274815,0.001875,0.642628,-0.654290,0.214574,-0.293547,0.130087,-0.229508,-0.694710,0.137633,0.656480,0.133304,0.095248,0.354127,-1.056880,0.703586,0.247048,-0.022042,-0.129785,-0.541663,0.394466,-0.488587,-0.364698,-0.332798
3,1416524797,-0.647013,0.180865,-0.489965,1.114732,0.094578,-1.057976,0.426130,0.370656,0.029934,-0.052134,-0.753881,1.078903,0.331643,1.423183,-0.517949,0.582537,0.093206,-0.165433,0.220710,-0.095140,1.147847,-0.736902,-0.108050,-0.474030,-0.693926,-0.531537,-0.734595,-0.384251,-0.357388,-0.007252,0.000105,-0.435453,-0.518411,-0.730604,0.546070,-0.451638,0.519674,0.358086,0.968112,1.078118,0.482283,-0.174446,0.061444,-0.287109,-0.764890,-0.364018,0.379815,0.985719,0.818427,0.459480,-1.393237,0.826720,-0.181059,-0.517559,-0.518846,-0.300878,0.556808,0.125865,-0.374460,-0.805402
4,0062024035,-0.246244,1.039363,-0.610816,0.225927,-0.403030,-0.584747,-0.386077,-0.967948,-0.154100,-0.325529,-0.431129,1.540700,0.826782,0.863193,0.328848,-0.332890,0.290573,-0.480421,-0.046965,-0.062864,-0.179505,1.486357,0.064144,-0.075986,0.667894,-0.938478,-1.189211,-0.207569,0.220822,0.454036,-0.073526,0.604913,0.778136,-1.200414,0.042461,-0.526114,0.584528,0.318058,0.402939,-0.169787,0.983707,0.136155,-1.075766,0.217211,-0.542643,0.429675,1.248992,0.027751,0.279038,-1.182502,-0.806256,1.326736,0.057221,-0.672005,-1.045173,0.188045,-0.192922,-1.414888,0.497852,-0.373606


In [ ]:
df.to_csv("/content/drive/MyDrive/ProjetML/title_fastText.csv", sep = "|", index = False)

# FastText on Author

### Information preparation

In [ ]:
df_books.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
#I only use the not empty book_author lines, and english books
df1 = df_books[df_books["book_language"] == "en"].copy()
lines_OK = df1[df1.book_author.isna() == False]
df1 = df1["book_author"]

print("len(df_books)", len(df_books))
print("len(df1)", len(df1))
print("len(lines_OK)", len(lines_OK), '\n')
df1[1000:1005]

len(df_books) 105508
len(df1) 93808
len(lines_OK) 92875 



1010         Alyson Noël
1011       Louise L. Hay
1013    Richard Matheson
1014       John Sandford
1015      Michael Chabon
Name: book_author, dtype: object

In [ ]:
sent = []

for i in lines_OK.index:
    #I get sentence inside book_title column, I remove final '.' and separate all words
    w = df1[i].replace(".", " ")
    w = w.split()

    #I remove stopwords, words of length < 2 and '...'
    ww = [word.strip(',') for word in w if len(word) > 2]

    #I reconstruct a sentence with the remaining words, separated with ','
    sent = sent + [ww]

In [ ]:
#list of lists creation: 1 list for each book
print(len(sent))
sent[15000:15005]

92875


[['Sid', 'Fleischman'],
 ['Patricia', 'Leitch'],
 ['Fred', 'Grove'],
 ['Theodore', 'Taylor'],
 ['Lynne', 'Reid', 'Banks']]

### Training

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 2
min_word = 1
down_sampling = 1e-2

In [ ]:
#It takes about 2min

#window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that I want to create skip-gram model.

ft_model_authOnly = FastText(sent,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
ft_model_authOnly.save("/content/drive/MyDrive/ProjetML/FastText_authOnly.model")

### Save embeddings

In [ ]:
df_fastText_authOnly = df_books.copy()
print("len(df_fastText_authOnly)", len(df_fastText_authOnly))

#I only embeds title of english books
df_fastText_authOnly = df_fastText_authOnly[df_fastText_authOnly["book_language"] == "en"]
print("len(df1)", len(df_fastText_authOnly))

#I only embeds not empty book_author liness (same selection as previously for training)
lines_OK = df_fastText_authOnly[df_fastText_authOnly.book_author.isna() == False]
print("len(lines_OK)", len(lines_OK), '\n')

len(df_fastText_authOnly) 105508
len(df1) 93808
len(lines_OK) 92875 



In [ ]:
#It takes about 30min

#Creation of 60 columns for author embeddings
col = ["c_" + str(i) for i in range(60)]
#I will save ISBN and author embeddings in a new dataframe
df = pd.DataFrame(columns = ["isbn"] + col)

for i in range(len(lines_OK.index)):

  #a tokenized sentences
  sent_auth = sent[i]

  if (len(sent_auth) > 0):

    #find the embedding (vector of 60) for all words inside tokenized sentence
    try :
      words_emb = [ft_model_authOnly.wv[w] for w in sent_auth]
    except Exception as e:
      #sometime name of author are not written in english (even if book's language is "en")
      print("pb with sent_auth: ", sent_auth, e)
    else:
      emb = [0.0]*60

      #Perform the sum of each words embeddings
      for word in words_emb:
        l = len(word)
        #sum of embedding of each words
        emb = [emb[k] + word[k] for k in range(l)]

      emb = [emb[k] / len(words_emb) for k in range(l)]

      # Saving
      df.loc[i] = [str(df_fastText_authOnly.isbn[lines_OK.index[i]])] + emb


In [ ]:
print(words_emb)
print(len(words_emb))
print(len(df))
df.head()

[array([ 1.0466986e+00, -7.7145344e-01, -1.1388736e+00,  2.9503745e-01,
        5.4668933e-01, -1.9650407e+00, -2.5195472e+00,  2.6679583e+00,
       -2.6305133e-01,  1.4337145e+00,  3.6253357e-01,  2.6310573e+00,
       -8.8804513e-01,  3.5839596e-01,  6.8032980e-01,  1.1799196e+00,
       -8.2390308e-02,  2.4319985e+00,  6.2914900e-02, -7.3310626e-01,
        1.2216159e+00, -4.1514920e-04, -6.9051474e-01,  9.4194525e-01,
       -1.8151350e-01, -9.8386163e-01,  7.2151432e-03, -2.0796137e+00,
       -1.0818155e+00, -6.0946250e-01, -6.9402796e-01, -1.3677983e-01,
        2.6938765e+00,  2.7152026e-01,  1.6154886e+00, -1.2229954e+00,
       -2.2931702e+00,  5.9110314e-01,  1.1180580e+00,  1.5133610e+00,
        5.6969178e-01, -1.1234399e+00, -3.4321889e-02, -1.8024451e-01,
       -9.5949060e-01, -7.9564023e-01,  3.3817950e-01,  6.8118550e-02,
        1.1257071e+00,  7.9887158e-01, -1.3495807e-01, -9.9184847e-01,
        7.0569414e-01,  1.1301403e+00,  1.9749242e+00, -3.1320906e-01,
     

,isbn,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,0439023483,0.342431,0.297825,0.041389,0.318763,0.756700,-1.011396,-0.327488,4.359544,0.662114,0.292969,-1.562622,0.767075,-0.378741,0.493275,0.226446,-0.633773,-2.664377,0.808544,0.273756,-0.826814,0.758020,1.916958,-1.282348,0.788074,-0.994454,1.112676,0.297506,-2.641000,-0.710001,2.184157,0.040942,0.463229,2.292549,0.579022,0.785905,-1.379118,-0.924367,-0.415781,1.567474,1.657033,-0.720935,-1.475425,0.631479,-0.697283,0.256653,-1.832851,0.873207,-1.199664,0.986724,0.883189,0.239872,-0.122116,-1.573812,1.441412,0.696378,-0.541776,0.127434,0.005944,0.862448,0.026934
1,0316015849,-0.454511,0.055543,-0.287347,-0.837172,-0.526596,-0.655587,-0.875121,3.504816,-0.393957,1.166422,-0.518221,0.989242,-0.744715,0.343235,1.155785,-0.758849,-1.138831,1.907541,0.182787,-0.417936,1.804762,-0.231035,-1.248132,0.846120,-1.164794,1.227368,-1.295113,-1.138516,0.581425,2.863055,0.078590,0.546591,0.657815,0.561831,-0.298749,-0.484231,-1.008960,0.588761,1.004155,0.723218,-0.043640,0.288447,0.144546,0.722903,-0.501195,-0.910611,0.052724,-0.459757,1.018415,1.205273,0.504914,0.154590,-0.958867,0.402974,0.469348,1.249710,-0.487583,0.767349,0.460011,-0.200559
2,0525478817,-1.153355,-1.809237,-1.157749,0.079552,-0.147029,-1.150021,-1.675198,1.868109,-1.554916,1.494470,0.001662,0.907905,-1.843146,-0.277120,-0.402535,0.323738,0.256219,1.483350,-0.289515,-0.851747,1.330736,0.174346,-2.573746,-0.952620,-0.981183,-0.187485,0.024682,-0.108759,-1.872074,2.320355,0.020613,-0.539296,0.715753,0.465317,-0.494404,-0.585286,-0.359360,1.263346,0.285298,0.243991,-0.992017,-0.584551,-0.053135,0.230913,-0.048448,-0.577313,0.343338,-1.092123,-1.618673,1.352470,0.499026,0.771437,-1.566328,0.866268,-0.313696,0.812361,0.657456,0.619461,1.317496,0.218175
3,1416524797,-2.103752,0.768707,-0.618245,1.169080,0.602093,-1.296557,-0.915950,4.369274,-0.920109,0.935629,-0.898917,1.807655,-2.244977,0.454944,-0.769482,-0.003694,-0.238882,2.553699,0.612737,-0.159067,-0.961502,1.275587,1.261477,0.890394,-1.482434,1.973676,0.829303,-0.822820,0.053147,1.563887,1.168144,0.618269,1.508263,0.572795,-0.135657,0.482000,-0.248704,1.027536,-0.018751,0.523506,-0.743749,-1.618760,-0.136217,1.302089,0.797350,-1.647253,0.092957,-0.266230,-1.186900,2.120644,0.450062,0.830490,0.231699,0.296176,0.407748,1.778338,0.776784,1.155313,0.868761,-0.232059
4,0062024035,-0.609772,0.402986,-0.621646,0.111180,0.963444,-0.481492,-0.717680,2.712170,-0.603851,0.206139,-2.127428,0.834950,-1.706133,0.620978,0.595586,-0.472663,-0.430246,1.378117,0.760914,-0.470372,2.145526,0.804606,-0.470122,0.057988,-0.721162,0.398941,0.430524,-1.085778,-0.663373,1.490663,0.335501,-0.392221,1.052583,1.439919,0.677476,-0.805209,-1.025358,0.271716,-0.084871,0.410879,0.473611,-0.248139,0.926682,0.746023,-0.304264,-0.799315,0.586745,0.135809,0.088280,0.909821,0.251135,0.378479,0.322816,-0.101207,0.457223,0.871265,0.163264,1.127235,0.876887,-0.538869


In [ ]:
df.to_csv("/content/drive/MyDrive/ProjetML/authOnly_fastText.csv", sep = "|", index = False)

# FastText on Categories
Input data are then : 
*   Category
*   Category_other
*   author_genres
*   author_genres_other


### Information preparation

In [ ]:
#I keep only few columns to create the context of training

#I only use english books, with at least one elements among columns is not empty
df1 = df_books[df_books["book_language"] == "en"].copy()
lines_OK = df1[(df1.Category.isna() == False) | \
               (df1.Category_other.isna() == False) | \
               (df1.author_genres.isna() == False) | \
               (df1.author_genres_other.isna() == False)]
df1 = df1[["Category", "Category_other", "author_genres", "author_genres_other"]]

print("len(df_books)", len(df_books))
print("len(df1)", len(df1))
print("len(lines_OK)", len(lines_OK), '\n')
df1.tail()

len(df_books) 105508
len(df1) 93808
len(lines_OK) 90141 



,Category,Category_other,author_genres,author_genres_other
105502,arthurian rom,NaN,science fict,fantasi
105504,mothers and daught,NaN,poetri,NaN
105505,domestic fict,NaN,NaN,NaN
105506,juvenile nonfict,NaN,disambigu,NaN
105507,fiction,NaN,NaN,NaN


In [ ]:
for i in lines_OK.index:
    #Category: in case where several words, I separate all words by ','
    if (str(df1.Category[i]) != "nan"):
      w = df1.Category[i].split()
      #I reconstruct a sentence with the remaining words, separated with ','
      df1.Category[i] = ','.join(w)

    #same for Category_other
    if (str(df1.Category_other[i]) != "nan"):
      w = df1.Category_other[i].split()
      df1.Category_other[i] = ','.join(w)

    #same for author_genres
    if (str(df1.author_genres[i]) != "nan"):
      w = df1.author_genres[i].split()
      df1.author_genres[i] = ','.join(w)

    #same for author_genres_other
    if (str(df1.author_genres_other[i]) != "nan"):
      w = df1.author_genres_other[i].split()
      df1.author_genres_other[i] = ','.join(w)

df1[1000:1005]

,Category,Category_other,author_genres,author_genres_other
1010,"young,adult,fict",NaN,"young,adult",NaN
1011,chemistri,psycholog,religion,spiritu
1013,"horror,tal",NaN,"science,fict",fantasi
1014,fiction,NaN,mysteri,thriller
1015,fiction,NaN,literatur,fiction


In [ ]:
#All those columns informations must be joined inside one column
#str(x) != "nan":     but I join only if column word is not empty
#x.split():           sentences in colum 'book_title' are cut in words
#','.join(x.split()): all those words are joined with (',') to be transformed after in list of list
def WordToJoin(w):
  ww = [x for x in w if str(x) != "nan"]
  return ','.join(ww)

df2 = df1.apply(lambda x: WordToJoin(x), axis = 1)
df2.head()

0                            juvenile,fict
1    young,adult,fict,science,fict,fantasi
2          juvenile,fict,literatur,fiction
3                 fiction,mysteri,thriller
4               juvenile,fict,science,fict
dtype: object

In [ ]:
#a specific data frame is created with all this train context
df_clean = pd.DataFrame({'clean': df2})
df_clean.head()

,clean
0,"juvenile,fict"
1,"young,adult,fict,science,fict,fantasi"
2,"juvenile,fict,literatur,fiction"
3,"fiction,mysteri,thriller"
4,"juvenile,fict,science,fict"


In [ ]:
#list of lists creation: 1 list for each book
sent = [row.split(',') for row in df_clean["clean"]]

print(len(sent))
sent[0:5]

93808


[['juvenile', 'fict'],
 ['young', 'adult', 'fict', 'science', 'fict', 'fantasi'],
 ['juvenile', 'fict', 'literatur', 'fiction'],
 ['fiction', 'mysteri', 'thriller'],
 ['juvenile', 'fict', 'science', 'fict']]

### Training

In [ ]:
#The training has been performed without Colab
from gensim.models.fasttext import FastText

In [ ]:
#The training has been performed without Colab
embedding_size = 60
window_size = 3
min_word = 1
down_sampling = 1e-2

In [ ]:
#The training has been performed without Colab (problem of session crach)
#------------------------------------------------------------------------

#window_size :
#       is the size of the number of words occurring before and after 
#       the word based on which the word representations will be learned

#min_word :
#       which specifies the minimum frequency of a word in the corpus 
#       for which the word representations will be generated.

#sg :
#       A value of 1 specifies that I want to create skip-gram model.

ft_model_4cat = FastText(sent,
                      size      = embedding_size,
                      window    = window_size,
                      min_count = min_word,
                      sample    = down_sampling,
                      sg        = 1,
                      iter      = 100)

In [ ]:
ft_model_4cat.save("/content/drive/MyDrive/ProjetML/FastText_4cat.model")

### Save embeddings

In [ ]:
from gensim.models.fasttext import FastText
ft_model_4cat = FastText.load("/content/drive/MyDrive/ProjetML/FastText_4cat.model")

In [ ]:
df_fastText_4cat = df_books.copy()
print("len(df_fastText_4cat)", len(df_fastText_4cat))

#I only use the not empty categories information, and english books (same selection as previously for training)
df_fastText_4cat = df_fastText_4cat[df_fastText_4cat["book_language"] == "en"]
print("len(df1)", len(df_fastText_4cat))

lines_OK = df_fastText_4cat[(df_fastText_4cat.Category.isna() == False) \
                            | (df_fastText_4cat.Category_other.isna() == False) \
                            | (df_fastText_4cat.author_genres.isna() == False) \
                            | (df_fastText_4cat.author_genres_other.isna() == False)]

print("len(lines_OK)", len(lines_OK), '\n')

len(df_fastText_4cat) 105508
len(df1) 93808
len(lines_OK) 90141 



In [ ]:
#It takes about 30min

#Creation of 60 columns for categories embeddings
col = ["c_" + str(i) for i in range(60)]
#I will save ISBN and categories embeddings in a new dataframe
df = pd.DataFrame(columns = ["isbn"] + col)

for i in range(len(lines_OK.index)):

  #a tokenized sentences
  sent_4cat = sent[i]

  if (len(sent_4cat) > 0):

    #find the embedding (vector of 60) for all words inside tokenized sentence
    words_emb = [ft_model_4cat.wv[w] for w in sent_4cat]

    emb = [0.0]*60

    #Perform the sum of each words embeddings
    for word in words_emb:
      l = len(word)
      #sum of embedding of each words
      emb = [emb[k] + word[k] for k in range(l)]

    emb = [emb[k] / len(words_emb) for k in range(l)]

    # Saving
    df.loc[i] = [str(df_fastText_4cat.isbn[lines_OK.index[i]])] + emb


In [ ]:
print(words_emb)
print(len(words_emb))
print(len(df))
df.head()

[array([ 0.40499994,  0.47939864,  0.2958442 ,  0.8038185 , -0.10657068,
        0.2701805 ,  0.44238853,  0.03187859, -0.19643934,  0.98922396,
        0.18098779,  0.07225538, -0.5448017 , -0.06013599,  0.1809506 ,
        0.7052224 ,  0.1434151 ,  0.07236414,  0.7402752 ,  1.394313  ,
        0.2726269 ,  0.6115681 ,  0.36438802,  1.0322398 , -0.24358523,
        0.358138  ,  0.2325866 ,  0.92510515,  1.2976094 ,  2.1014094 ,
       -0.34853163, -1.1227388 ,  0.4329388 ,  0.7479089 , -0.4311141 ,
        0.80914   , -0.4071753 ,  0.43168882,  1.3476216 ,  1.0586842 ,
        0.85741985,  0.2941847 ,  0.23232909,  0.15334435, -0.4861662 ,
       -0.18968755,  0.7955024 ,  0.17289881,  0.0203477 ,  0.6590071 ,
        0.29923072,  0.47384176, -1.2813555 , -0.31583762, -0.7455307 ,
        0.32210937, -0.07206117,  0.798324  ,  0.19499321,  0.20836182],
      dtype=float32)]
1
90141


,isbn,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,0439023483,0.312050,0.158412,0.387731,0.090899,-0.285965,0.343511,0.223471,-0.340556,-0.110160,0.396203,-0.282604,-0.197878,-0.185222,-0.038657,-0.045248,-0.075658,-0.045496,0.142062,1.123188,0.089430,0.127094,0.679097,0.521375,0.282981,0.126023,0.188324,0.374693,-0.482950,0.080204,0.184072,-0.468960,-0.435085,-0.300061,-0.015820,0.043474,-0.256620,0.218759,0.193508,0.524886,-0.221374,0.159251,0.176803,0.418329,0.508690,-0.259411,0.909242,-0.117321,-0.214536,0.007890,0.149371,0.160802,-0.256614,0.022174,0.090670,0.249283,-0.292247,-0.022732,0.209478,-0.031216,-0.132709
1,0316015849,0.254742,0.060327,-0.109539,-0.309455,-0.151214,0.385797,0.010280,-0.184530,0.018583,0.302963,-0.073787,-0.364000,-0.068532,0.139073,-0.259777,-0.348455,-0.041521,-0.119068,0.729936,0.052797,0.182703,1.017126,0.533827,0.172156,-0.164359,0.300084,0.049345,-0.094257,-0.088710,0.129054,-0.156019,-0.062156,-0.337189,0.196943,-0.267655,-0.005214,0.159620,0.304268,0.371932,-0.159900,0.319063,0.226690,0.539549,0.209176,-0.395906,0.762893,-0.690063,-0.112445,0.127332,0.370009,0.440636,-0.063268,0.086089,0.374464,0.003211,0.069026,-0.059511,0.327391,0.078629,-0.120300
2,0525478817,0.366934,0.091443,0.132765,0.129768,-0.181872,0.315206,0.117245,-0.355384,-0.092908,0.265384,-0.081584,-0.195244,-0.084385,0.178171,-0.316042,-0.002317,-0.007217,0.027824,0.746478,-0.129341,0.103396,0.682931,0.174143,0.358069,-0.097498,0.308164,0.169619,-0.359177,-0.007713,-0.008083,-0.360514,-0.342991,-0.140916,0.096451,0.021700,-0.264332,0.250770,0.224414,0.314265,-0.200613,0.120835,0.320563,0.224461,0.333079,-0.220076,0.594831,-0.178671,-0.050414,-0.204611,0.010572,0.090687,-0.062060,0.056913,-0.094466,-0.143608,-0.044100,-0.029729,0.218964,0.019288,0.048844
3,1416524797,0.118512,-0.081398,-0.256817,0.087609,-0.128643,0.589855,-0.005112,-0.315501,0.198383,0.386195,0.372035,0.056572,0.311377,0.342454,-0.519596,0.051780,0.321420,-0.328252,0.325039,0.154926,0.038537,0.986481,-0.322208,0.353785,-0.225548,-0.040931,0.204153,-0.083519,0.440369,-0.114197,-0.316922,-0.195214,-0.061428,0.012139,-0.304002,-0.303446,0.466622,0.251725,0.104868,0.105890,-0.197734,0.522194,0.400878,0.481159,-0.063575,0.228800,-0.299319,-0.013327,-0.479204,0.259896,-0.113262,0.144578,0.361504,0.057417,-0.110583,0.390464,-0.351632,0.391705,0.049350,0.529363
4,0062024035,0.240475,0.189523,0.147694,0.040850,-0.226454,0.329235,0.267512,-0.203096,-0.000052,0.390864,-0.131196,-0.282758,-0.220983,0.061065,-0.176746,-0.216055,0.083015,-0.005968,0.932381,0.113452,0.368699,0.689509,0.552407,0.319072,0.043016,0.299344,0.160784,-0.343707,-0.069567,0.096732,-0.226710,-0.307184,-0.371917,0.085235,-0.151377,-0.169994,0.144661,0.149316,0.452032,-0.230054,0.261233,0.252199,0.517902,0.376727,-0.306471,0.714501,-0.282183,-0.253579,-0.018433,0.266103,0.327499,-0.247227,0.075807,0.273640,-0.028109,-0.089047,-0.050536,0.281202,0.080389,-0.156915


In [ ]:
df.to_csv("/content/drive/MyDrive/ProjetML/4cat_fastText.csv", sep = "|", index = False)

# Train / Test with embeddings on matrix format

=> Only for the following embeddings :
*   FastText on AuthorCOntext
*   FastText on title
*   FastText on Author
*   FastText on Categories

Remark : For the other embeddings, similar process is performed under section 'Ordinal regression'
     

### Embedding matrix

In [ ]:
#FastText on Title
#df_fastText_title = pd.read_csv("/content/drive/MyDrive/ProjetML/title_fastText.csv", sep = "|")

#FastText on Author_Context :
# ["book_title", "book_author", "Category", "Category_other", "author_genres", "author_genres_other"]
df_fastText_auth = pd.read_csv("/content/drive/MyDrive/ProjetML/authorsContext_bis_fastText.csv", sep = "|")

In [ ]:
print(len(df_fastText_auth))
df_fastText_auth.head(2)

93808


,isbn,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,0439023483,-1.133914,0.287926,-0.340693,0.304958,0.298398,-0.173713,-0.143411,0.229791,0.161779,-0.452773,0.651183,-0.976731,-0.251009,0.249334,-0.425694,-0.049007,-0.170870,0.213249,0.627869,-0.062812,0.058515,-0.288496,0.009767,-0.097794,0.179403,-0.282955,0.197258,0.012673,0.048874,-0.422243,0.032915,-0.418987,-0.094094,-1.181253,0.203269,-0.195160,0.066798,0.305457,-0.251446,-0.330791,0.604022,-0.311264,0.439883,0.393421,-0.087169,-0.278171,-0.129530,0.384977,-0.419687,0.456741,0.174104,0.716973,-0.182069,0.037079,-0.085198,-0.071911,0.257613,0.467097,0.058294,0.152557
1,0316015849,-0.661967,0.020827,-0.102629,0.656167,-0.522500,-0.686821,-0.081805,-0.155748,0.310289,-1.080961,0.280845,-1.200535,-1.021454,0.434017,-0.708987,0.037595,-0.258162,0.517609,0.635528,-0.027594,0.039133,-0.095607,0.306347,0.124747,0.379338,0.349878,-0.797516,0.003671,0.246132,-0.511566,-0.860476,-1.117019,-0.060744,-0.885507,0.515735,-0.011868,-0.119200,-0.016419,-0.142642,-0.626561,0.135914,-0.473242,0.192619,0.394811,0.276300,-0.479259,0.379815,0.882635,-0.855744,-0.262017,0.313448,0.648078,0.309740,-0.321710,0.097637,-0.480251,-0.052613,0.416854,0.032663,0.339096


### Merge

In [ ]:
#BookCrossing provided ratings
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

In [ ]:
print(len(df_ratings))
print(df_ratings["Book-Rating"].value_counts(sort = False))
df_ratings.head(1)

1149780
0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: Book-Rating, dtype: int64


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [ ]:
  #I merge both dataframe (embeddings and ratings) according to ISBN  which is a common column
  df_fastText_auth = pd.merge(df_ratings, df_fastText_auth, left_on = "ISBN", right_on = "isbn")  
  print(len(df_fastText_auth))
  df_fastText_auth.head()

83569


,User-ID,ISBN,Book-Rating,isbn,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,276725,034545104X,0,034545104X,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353
1,2313,034545104X,5,034545104X,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353
2,6543,034545104X,0,034545104X,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353
3,8680,034545104X,5,034545104X,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353
4,10314,034545104X,9,034545104X,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353


### Columns selection

In [ ]:
col = ["User-ID"] + ["c_" + str(i) for i in range(60)]

In [ ]:
  #I just remove the redundant and useless columns after the merge
  X_data = df_fastText_auth[col]
  print(len(X_data))
  X_data.head(2)

83569


,User-ID,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,c_40,c_41,c_42,c_43,c_44,c_45,c_46,c_47,c_48,c_49,c_50,c_51,c_52,c_53,c_54,c_55,c_56,c_57,c_58,c_59
0,276725,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353
1,2313,-0.689544,0.327739,0.281971,0.992131,-0.126279,-0.019933,-0.51757,0.307573,0.345663,-0.428409,-0.010448,-0.441609,0.114402,0.016148,-0.452704,0.118352,-0.448979,0.53418,0.291999,0.231614,-0.06471,0.369374,-0.057884,0.68392,0.388908,0.119315,-0.349792,-0.181237,0.454748,-0.364846,0.290088,-0.996198,0.259048,-0.644573,0.312212,-0.698993,-0.039816,0.39935,-0.140139,0.117426,-0.134278,-0.250076,0.64603,1.024134,-0.319981,-0.623573,0.235108,0.666163,-0.625195,0.467862,0.239082,0.675744,0.657425,-0.274329,-0.123018,-0.303799,-0.1408,-0.062162,-0.255123,0.320353


### Creation of mat_X_data

In [ ]:
#In order to normalize those User-ID...even if bad idea to keep them
MIN = min(df_fastText_auth["User-ID"])
MAX = max(df_fastText_auth["User-ID"])
print(MIN, MAX)

8 278851


In [ ]:
def vectfastText(l, s):
  """
  Inputs :
    l is the current merged (User-ID & embedding) line to normalized, and after columns selection
    s is the embedding size
  Output : the same current line, but with :
    User-ID normalized
    a vector format
  """

  #s is the size of embedding (60 for fastText)
  b = np.zeros(s+1)

  #1st copy of User-ID  & normalisation
  b[0] = (l[0]-MIN)/(MAX-MIN)

  #Then handling of float in each columns of l (a line of the merge data frame)
  b[1:] = [float(l[i]) for i in range(1, s+1)]
  
  return b

In [ ]:
print(vectfastText(X_data.iloc[0], 60).shape)
vectfastText(X_data.iloc[0], 60)

(61,)


array([ 0.99237564, -0.6895437 ,  0.3277392 ,  0.28197122,  0.99213111,
       -0.12627933, -0.01993306, -0.51757024,  0.30757296,  0.34566287,
       -0.42840915, -0.01044847, -0.44160875,  0.11440196,  0.01614794,
       -0.45270438,  0.11835214, -0.44897868,  0.53417971,  0.29199894,
        0.23161354, -0.0647097 ,  0.36937358, -0.05788426,  0.68392011,
        0.38890841,  0.11931475, -0.34979196, -0.18123682,  0.45474767,
       -0.36484604,  0.29008785, -0.99619838,  0.25904838, -0.64457256,
        0.31221218, -0.69899347, -0.03981559,  0.39935   , -0.1401387 ,
        0.11742553, -0.13427791, -0.25007566,  0.64602979,  1.02413405,
       -0.31998064, -0.62357253,  0.23510752,  0.66616345, -0.62519519,
        0.46786245,  0.23908152,  0.67574397,  0.65742459, -0.27432858,
       -0.12301763, -0.30379925, -0.14079995, -0.06216162, -0.25512299,
        0.32035297])

In [ ]:
def matfastText(l, s, df):
  """
  Inputs :
    l is the number of df lines to normalize and transform into vector
    s is the embedding size
    df is the merged dataframe (User-ID & embedding)
  Output : the same df, but with 
    User-ID normalized
    Matrix format
  """

  #number of columns of the matrix returned
  #for example fastText embedding of s1 = 60 
  #          + User-ID
  #l is the size of the merged data frame (lines number)
  m = np.zeros((l, (s+1)))
  
  ind = 0
  for i in range(l):
      #all the vectors embeddings are put inside a matrix line
      m[ind] = vectfastText(df.iloc[i], s)
      ind = ind + 1
  
  return m

In [ ]:
mat_X_data = matfastText(len(X_data), 60, X_data)
mat_X_data.shape

(83569, 61)

### Labels selection

In [ ]:
  #I use again the merged dataset according to ISBN (embeddings and ratings), created before columns selection 
  Y_data = df_fastText_auth["Book-Rating"]
  print(len(Y_data))
  Y_data.head()

83569


0    0
1    5
2    0
3    5
4    9
Name: Book-Rating, dtype: int64

In [ ]:
#Labels (eg. ratings) are put on np.ndarray format
def vectLabels(l, df):
  m = np.zeros((l, 1), dtype = int)
  
  ind = 0
  for i in range(l):
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [ ]:
vect_Y_data = vectLabels(len(Y_data), Y_data)
vect_Y_data.shape

(83569, 1)

### Train / Test split

In [ ]:
#mat_X_data: matrix obtained after 
#               ratings & embeddings merge -> columns selection -> Normalisation of User-ID & matrix format
#vect_Y_data: labels transformed in matrix format                       
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [ ]:
print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

X_train.shape (66855, 61)
X_test.shape (16714, 61) 

y_train.shape (66855, 1)
y_test.shape (16714, 1)


# Balancing classes
Some trials to balance classes

### Binary ratings

In [ ]:
#BookCrossing provided ratings
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")
print(len(df_ratings))
df_ratings.head(1)

1149780


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [ ]:
df_BinRatings = df_ratings.copy()

#Very unbalanced classes
df_BinRatings["Book-Rating"].value_counts(sort = False )

0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: Book-Rating, dtype: int64

In [ ]:
#Ratings (or 'classes') 0 are put to 15 :
#  it's an artificial way to put them temporarily away

#Then the two binary classes are created :
# 1, 2, 3 and 4 ratings become class 0
# 7, 8, 9 and 10 ratings become class 1

#Then a part of 0 previous rating will be used to balance the binary classes
#307 000 lines of separated 15 values will be taken in random order to fill the binary classe 0
df_BinRatings["Book-Rating"] = df_BinRatings["Book-Rating"].map({0: 15, 1: 0, 2: 0, 3: 0, 4: 0, 7: 1, 8: 1, 9: 1, 10: 1})
df_BinRatings["Book-Rating"].value_counts(sort = False)

0.0      19429
15.0    716109
1.0     326344
Name: Book-Rating, dtype: int64

In [ ]:
#Values 15 (old 0 values) are selected...
df_BinRatings15 = df_BinRatings[df_BinRatings["Book-Rating"] == 15]
print(len(df_BinRatings15))
df_BinRatings15.head()

716109


,User-ID,ISBN,Book-Rating
0,276725,034545104X,15.0
2,276727,0446520802,15.0
5,276733,2080674722,15.0
10,276746,0425115801,15.0
11,276746,0449006522,15.0


In [ ]:
#... and put in random order
df_BinRatings15 = df_BinRatings15.sample(frac = 1, random_state=10).reset_index(drop = True)
print(len(df_BinRatings15))
df_BinRatings15.head()

716109


,User-ID,ISBN,Book-Rating
0,265202,0385471548,15.0
1,78974,8420400963,15.0
2,190807,0671042475,15.0
3,252695,0843950757,15.0
4,190708,044015393X,15.0


In [ ]:
#then only the first 307000 lines are kept in order to balance our future binary classes
df_BinRatings15 = df_BinRatings15[:307000]
print(len(df_BinRatings15))
df_BinRatings15.head()

307000


,User-ID,ISBN,Book-Rating
0,265202,0385471548,15.0
1,78974,8420400963,15.0
2,190807,0671042475,15.0
3,252695,0843950757,15.0
4,190708,044015393X,15.0


In [ ]:
#Then initial values 0 are put again
df_BinRatings15["Book-Rating"] = df_BinRatings15["Book-Rating"].map({15: 0})
print(len(df_BinRatings15))
df_BinRatings15.head()

307000


,User-ID,ISBN,Book-Rating
0,265202,0385471548,0
1,78974,8420400963,0
2,190807,0671042475,0
3,252695,0843950757,0
4,190708,044015393X,0


In [ ]:
#know I will concatenate new 0 classe...
df_BinRatings0 = df_BinRatings[df_BinRatings["Book-Rating"] == 0]
print(len(df_BinRatings0))
df_BinRatings0.head()

19429


,User-ID,ISBN,Book-Rating
3,276729,052165615X,0.0
49,276762,3453213025,0.0
55,276762,N3453124715,0.0
57,276768,9057868059,0.0
241,276853,8420457477,0.0


In [ ]:
#...with new 1 classe...
df_BinRatings1 = df_BinRatings[df_BinRatings["Book-Rating"] == 1]
print(len(df_BinRatings1))
df_BinRatings1.head()

326344


,User-ID,ISBN,Book-Rating
6,276736,3257224281,1.0
8,276744,038550120X,1.0
9,276745,342310538,1.0
16,276747,0060517794,1.0
19,276747,0671537458,1.0


In [ ]:
#...and with previous treated 0 values
df_ratings_new = pd.concat([df_BinRatings15, df_BinRatings0, df_BinRatings1], ignore_index = True, axis = 0)	
print(len(df_ratings_new))
df_ratings_new.head()

652773


,User-ID,ISBN,Book-Rating
0,265202,0385471548,0.0
1,78974,8420400963,0.0
2,190807,0671042475,0.0
3,252695,0843950757,0.0
4,190708,044015393X,0.0


In [ ]:
df_ratings_new["Book-Rating"].value_counts(sort = False )

0.0    326429
1.0    326344
Name: Book-Rating, dtype: int64

In [ ]:
df_ratings_new.to_csv("/content/drive/MyDrive/ProjetML/BX-Book-Ratings-Binary.csv", sep = "|", index = False)

### Only 5 classes (instead of 10)





In [ ]:
#BookCrossing provided ratings
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")
print(len(df_ratings))

#Very unbalanced classes
print(df_ratings["Book-Rating"].value_counts(sort = False))
df_ratings.head(1)

1149780
0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: Book-Rating, dtype: int64


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [ ]:
df_5Ratings = df_ratings.copy()

#Reduction of previous classes number by dividing values by 2
# then for example previous classes 2 and 3 become new 2 class
df_5Ratings["Book-Rating"] = np.floor(df_5Ratings["Book-Rating"] / 2)

df_5Ratings["Book-Rating"].value_counts(sort = False)

0.0    717879
2.0     59878
3.0    113381
4.0    171277
5.0     78610
1.0      8755
Name: Book-Rating, dtype: int64

# Ordinal regression

### Train / Test data creation
=> Only for the following embeddings :
*   Bert on books descriptions
*   FastText on books descriptions

Those embeddings have been saved as strings, so a specific pre-treatment is needed before using it.

Remark : For the other embeddings, a similar process is performed on previous section 'Train / Test with embeddings on matrix format'

##### DF loadings

In [ ]:
My_embedding = "bert"
#My_embedding = "fastText"

In [ ]:
#I get the bert embedding of book_description 
if (My_embedding == "bert"):
  #Bert on books descriptions
  df_bert = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_bert.csv", sep = "|")
  print(len(df_bert))

if (My_embedding == "fastText"):
  #FastText on books descriptions
  df_fastText = pd.read_csv("/content/drive/MyDrive/ProjetML/Books_description_fastText.csv", sep = "|")
  print(len(df_fastText))

105508


##### Merge

In [ ]:
#BookCrossing provided ratings
df_ratings = pd.read_csv(ratings, sep = ";", error_bad_lines=False, encoding="latin-1")

#Created ratings (from the BookCrossing ones), in a trial to balance classes (see previous 'Balancing classes')
#df_ratings = df_5Ratings
#df_ratings = pd.read_csv(ratings_bin, sep = "|", error_bad_lines=False, encoding="latin-1")

print(len(df_ratings))
print(df_ratings["Book-Rating"].value_counts(sort = False))
df_ratings.head(1)

1149780
0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: Book-Rating, dtype: int64


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0


In [ ]:
#Join of ratings and books data base, according to ISBN which is a common column

if (My_embedding == "bert"):
  df_bert = pd.merge(df_ratings, df_bert, left_on = "ISBN", right_on = "isbn")  
  print(len(df_bert))
  
  #I only use the not empty book_description lines, and english books
  lineOK = df_bert[(df_bert["book_description"].isna() == False) & \
                   (df_bert["book_language"] == "en") & \
                   (df_bert["description_bert"] != '0')]
  print(len(lineOK))

if (My_embedding == "fastText"):
  df_fastText = pd.merge(df_ratings, df_fastText, left_on = "ISBN", right_on = "isbn")  
  print(len(df_fastText))

  #I only use the not empty book_description lines, and english books
  lineOK = df_fastText[(df_fastText["book_description"].isna() == False) & \
                       (df_fastText["book_language"] == "en") & \
                       (df_fastText["description_fastFM"] != '0')]
  print(len(lineOK))

110801
63264


##### Columns selection

In [ ]:
#I just remove the redundant and useless columns after the merge
if (My_embedding == "bert"):
  X_data = df_bert[["User-ID", "description_bert"]]

if (My_embedding == "fastText"):
  X_data = df_fastText[["User-ID", "description_fastFM"]]

X_data.head()

,User-ID,description_bert
0,276725,"['-0.20497975', '0.032238785', '0.0620426', '-..."
1,2313,"['-0.20497975', '0.032238785', '0.0620426', '-..."
2,6543,"['-0.20497975', '0.032238785', '0.0620426', '-..."
3,8680,"['-0.20497975', '0.032238785', '0.0620426', '-..."
4,10314,"['-0.20497975', '0.032238785', '0.0620426', '-..."


##### Strings of embeddings transformed into float

In [ ]:
#Those values will be used in order to normalize User-ID
if (My_embedding == "bert"):
  s = 768
  MIN = min(df_bert["User-ID"])
  MAX = max(df_bert["User-ID"])
  print("min:", MIN)
  print("max:", MAX)

if (My_embedding == "fastText"):
  s = 60
  MIN = min(df_fastText["User-ID"])
  MAX = max(df_fastText["User-ID"])
  print("min:", MIN)
  print("max:", MAX)


min: 8
max: 278854


In [ ]:
#Transformation of a X_data line into a vector 
#because bert embedding has been saved as a list of strings

def vectBertfastText(l, s):
  """
  Inputs :
    l is the current merged (User-ID & embedding) line to normalize, and after columns selection
    s is the embedding size
  Output : the same current line, but with :
    User-ID normalized
    a vector format
  """

  #for example bert embedding of s = 768 and User-ID
  b = np.zeros(s+1)

  #1st copy of User-ID  & normalisation
  b[0] = (l[0]-MIN)/(MAX-MIN)

  #Then handling of list of strings
  v = l[1].split("'")
  b[1:] = [float(v[i]) for i in range(1, len(v), 2)]
  
  return b

In [ ]:
res = vectBertfastText(X_data.iloc[1], s)
print("\n", len(res))
res[0:5]


 769


array([ 0.00826621, -0.20497975,  0.03223878,  0.0620426 , -0.38984668])

In [ ]:
#treatment of all lines of X_data such that X_data.book_description is not empty

def matBertfastText(l, s, df):
  """
  Inputs :
    l are the df lines to normalize and transform into vector
    s is the embedding size
    df is the merged dataframe (User-ID & embedding)
  Output : the same df, but with 
    User-ID normalized
    Matrix format
  """

  #number of columns of the matrix returned
  #for example fastText embedding of s = 60 
  #          + User-ID
  #len(l) is the size of the merged data frame (lines l to handle)  
  m = np.zeros((len(l), (s+1)))
  
  ind = 0
  for i in l.index:
      #print(df.iloc[i])
      m[ind] = vectBertfastText(df.iloc[i], s)
      ind = ind + 1
  
  return m

In [ ]:
#It takes about 1min for about 75 000 lines
mat_X_data = matBertfastText(lineOK, s, X_data)
mat_X_data.shape

(63264, 769)

##### Labels lines selection

In [ ]:
#I use again the merged dataset according to ISBN (embeddings and ratings), created before columns selection 
if (My_embedding == "bert"):
  Y_data = df_bert["Book-Rating"]

if (My_embedding == "fastText"): 
  Y_data = df_fastText["Book-Rating"] 

Y_data.head()

0    0
1    5
2    0
3    5
4    9
Name: Book-Rating, dtype: int64

In [ ]:
#Labels (eg. ratings) are put on np.ndarray format
#=> same lines as lines used for previous merge (User-ID & embedding datasets)
def vectLabels(l, df):
  m = np.zeros((len(l), 1), dtype = int)
  
  ind = 0
  for i in l.index:
      m[ind] = df.iloc[i]
      ind = ind + 1
  
  return m

In [ ]:
vect_Y_data = vectLabels(lineOK, Y_data)
vect_Y_data.shape

(63264, 1)

##### Train / Test split

In [ ]:
#mat_X_data: matrix obtained after 
#               ratings & embeddings datasets merge -> columns selection -> Normalisation of User-ID & matrix format
#vect_Y_data: labels transformed in matrix format   
(X_train, X_test, y_train, y_test) = train_test_split(mat_X_data, vect_Y_data,          
                                           test_size=.2, stratify = vect_Y_data)

In [ ]:
print("len(lineOK)", len(lineOK), "\n")

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape, "\n")

print("y_train.shape", y_train.shape)
print("y_test.shape", y_test.shape)

len(lineOK) 63264 

X_train.shape (50611, 769)
X_test.shape (12653, 769) 

y_train.shape (50611, 1)
y_test.shape (12653, 1)


##### imblearn
Trial to balance classes, but with imblearn library
Applied on train set only

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
Counter(np.squeeze(y_train))

Counter({0: 31513,
         1: 70,
         2: 118,
         3: 272,
         4: 385,
         5: 2137,
         6: 1550,
         7: 3306,
         8: 4639,
         9: 2878,
         10: 3743})

In [ ]:
#Test 1 : I try to manually put the new classes size

#New number of elements per classes: only OVER sampled classes
strategy = {1:3000, 2:3000, 3:3000, 4:3000, 5:3000, 6:3000}
over_under_sample = SMOTE(sampling_strategy = strategy)
X_smo, y_smo = over_under_sample.fit_resample(X_train, y_train)

#Then new number of elements per classes: only UNDER sampled classes
strategy = {0:3000, 7:3000, 8:3000, 9: 3000, 10:3000}
undersample = RandomUnderSampler(sampling_strategy = strategy)
X_smo_c, y_smo_c = undersample.fit_resample(X_smo, y_smo)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
Counter(np.squeeze(y_smo_c))

Counter({0: 3000,
         1: 3000,
         2: 3000,
         3: 3000,
         4: 3000,
         5: 3000,
         6: 3000,
         7: 3000,
         8: 3000,
         9: 3000,
         10: 3000})

In [ ]:
#Test 2 : I try an automatic resampling, when only 5 classes (see 'Balancing classes' for more details)

smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
Counter(np.squeeze(y_smt))

Counter({0: 23025, 1: 30545, 2: 27063, 3: 25283, 4: 24038, 5: 26544})

In [ ]:
#Test 3 : I try to manually put the new classes size, when only 5 classes (see 'Balancing classes' for more details)

#New number of elements per classes: only OVER sampled classes
strategy = {1:4000, 2:4000}
over_under_sample = SMOTE(sampling_strategy = strategy)
X_smo, y_smo = over_under_sample.fit_resample(X_train, y_train)

#Then new number of elements per classes: only UNDER sampled classes
strategy = {0:4000, 3:4000, 4:4000}
undersample = RandomUnderSampler(sampling_strategy = strategy)
X_smo_c, y_smo_c = undersample.fit_resample(X_smo, y_smo)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
Counter(y_smo_c)

Counter({0: 4000, 1: 4000, 2: 4000, 3: 4000, 4: 4000, 5: 3743})

In [ ]:
#Test 4 : I try to manually put the new classes size

#New number of elements per classes: only OVER sampled classes
strategy = {1:2000, 2:2000, 3:2000, 4:2000, 6: 2000}
over_under_sample = SMOTE(sampling_strategy = strategy)
X_smo, y_smo = over_under_sample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
Counter(y_smo)

Counter({0: 31513,
         1: 2000,
         2: 2000,
         3: 2000,
         4: 2000,
         5: 2137,
         6: 2000,
         7: 3306,
         8: 4639,
         9: 2878,
         10: 3743})

In [ ]:
#Then new number of elements per classes: only UNDER sampled classes
strategy = {0:2000, 7:2000, 8:2000, 9:2000, 10: 2000}
undersample = RandomUnderSampler(sampling_strategy = strategy)
X_smo_c, y_smo_c = undersample.fit_resample(X_smo, y_smo)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Counter(y_smo_c)

Counter({0: 2000,
         1: 2000,
         2: 2000,
         3: 2000,
         4: 2000,
         5: 2137,
         6: 2000,
         7: 2000,
         8: 2000,
         9: 2000,
         10: 2000})

In [ ]:
#Test 5 : I try to manually put the new classes size

#New number of elements per classes: only OVER sampled classes
strategy = {1:3000, 2:3000, 3:3000, 4:3000, 5:3000, 6: 3000}
over_under_sample = SMOTE(sampling_strategy = strategy)
X_smo2, y_smo2 = over_under_sample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
#Then new number of elements per classes: only UNDER sampled classes
strategy = {0:3000, 8:3000, 10: 3000}
undersample = RandomUnderSampler(sampling_strategy = strategy)
X_smo2_c, y_smo2_c = undersample.fit_resample(X_smo2, y_smo2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Counter(np.squeeze(y_smo2_c))

Counter({0: 3000,
         1: 3000,
         2: 3000,
         3: 3000,
         4: 3000,
         5: 3000,
         6: 3000,
         7: 3306,
         8: 3000,
         9: 2878,
         10: 3000})

In [ ]:
#Test 6 : I try an automatic resampling

#It takes about 15min for about 55 0000 lines
smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
Counter(np.squeeze(y_smt))

Counter({0: 23129,
         1: 31331,
         2: 31204,
         3: 30283,
         4: 30148,
         5: 26692,
         6: 27497,
         7: 25570,
         8: 24568,
         9: 25518,
         10: 25757})

In [ ]:
#Test 7 : I try another automatic resampling

#=> Not tested because looks like X_smt, y_smt
sampling_strategy = 0.1
smt2 = SMOTETomek(sampling_strategy)
X_smt2, y_smt2 = smt.fit_sample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=F

In [ ]:
Counter(np.squeeze(y_smt2))

Counter({0: 23191,
         1: 31352,
         2: 31192,
         3: 30257,
         4: 30164,
         5: 26753,
         6: 27481,
         7: 25546,
         8: 24607,
         9: 25507,
         10: 25787})

##### PCA

In [ ]:
#Inputs (see ssection 'imblearn') : X_smo2_c, y_smo2_c (balanced X_train, y_train), X_test, y_test
#Outputs (principal components) : X_train_pca, y_smo2_c, X_test_pca, y_test

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#PCA performs better after a normalisation
sc = StandardScaler()
X_smo2_c_temp = sc.fit_transform(X_smo2_c)
X_test_temp = sc.transform(X_test)

pca = PCA(n_components = 150)
X_train_pca = pca.fit_transform(X_smo2_c_temp)
X_test_pca = pca.transform(X_test_temp)

In [ ]:
print("Train - Before {0} / After {1}".format(X_smo2_c.shape, X_train_pca.shape))
print("Test - Before {0} / After {1}".format(X_test.shape, X_test_pca.shape))

Train - Before (33184, 769) / After (33184, 150)
Test - Before (12653, 769) / After (12653, 150)


In [ ]:
#Tests performed with different values for n_components through PCA
#With 10 components :  44% of variance explained
#With 20 components :  56% of variance explained
#With 30 components :  63% of variance explained
#With 50 components :  71% of variance explained
#With 100 components : 82% of variance explained
#With 150 components : 87% of variance explained
print(pca.explained_variance_ratio_.sum())

### Model training

##### Loading
2 models are tested :
*   LogisticAT : ordinal regression all-threshold  
*   OrdinalRidge : linear regression with L2 penalization, but the loss function is minus absolute mean (no more Least Square) 



In [ ]:
!pip install mord
from mord import LogisticAT, OrdinalRidge

  Created wheel for mord: filename=mord-0.6-cp37-none-any.whl size=6007 sha256=872688fce658347047f8496aaf071f04384e58d3e3d1591527b528e2f32abc2b
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


##### Unbalanced ratings

In [ ]:
#Example of inputs used

#X_smo2_c (imblearn(X_train)), y_smo2_c (imblearn(y_train)) are the 5th test to balance classes
#X_train_pca (PCA(X_smo2_c)), X_test_pca (PCA(X_test))

#Inputs for model training : X_train_pca, y_smo2_c
#Later inputs for model validation : X_test_pca, y_test

#It takes about 7 to 14 min 
LAT = LogisticAT(alpha=0., max_iter = 10000)
LAT.fit(X_train_pca, np.squeeze(y_smo2_c))

ORD = OrdinalRidge(alpha=0., max_iter = 10000)
ORD.fit(X_train_pca, np.squeeze(y_smo2_c))

OrdinalRidge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=10000,
             normalize=False, random_state=None, solver='auto', tol=0.001)

##### Binary ratings

In [ ]:
Counter(np.squeeze(y_train))

Counter({0: 18763, 1: 19466})

In [ ]:
#It takes about 7 to 14 min 
LAT = LogisticAT(alpha=0., max_iter = 10000)
LAT.fit(X_train, np.squeeze(y_train))

ORD = OrdinalRidge(alpha=0., max_iter = 10000)
ORD.fit(X_train, np.squeeze(y_train))

OrdinalRidge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=10000,
             normalize=False, random_state=None, solver='auto', tol=0.001)

### Model validation

##### Unbalanced ratings

In [ ]:
def ConfMat(y_pred, y_true, nb_classes):
  """ misclassified elements for each classes """
  
  #array with the number of misclassified elements for each classes
  compResTab = np.zeros(nb_classes)

  for i in range (len(y_true)):
    #The correct classe is y_true[i], so I increment its counter of misclassified if y_pred[i] is not the same
    compResTab[int(y_true[i])] = compResTab[int(y_true[i])] + (y_pred[i] != y_true[i])
  
  #Total number of element inside each classes
  p = Counter(np.squeeze(y_true))

  #Display of the misclassified number / percentage for each classes
  print("Class \tMisclassified \t    Percentage")
  for i in range(nb_classes):
     print("{0}\t{1}\t\t\t{2}".format(i, compResTab[i], round(100*(compResTab[i]/p[i]), 2)))
  print("\nTOTAL\t{0}\t\t\t{1}".format(compResTab.sum(), round(100*compResTab.sum()/len(y_true),2)))

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 10343,
         1: 25,
         2: 40,
         3: 91,
         4: 127,
         5: 699,
         6: 522,
         7: 1094,
         8: 1523,
         9: 968,
         10: 1282})

In [ ]:
#Example of inputs used

#X_smo2_c (imblearn(X_train)), y_smo2_c (imblearn(y_train)) are the 5th test to balance classes
#X_train_pca (PCA(X_smo2_c)), X_test_pca (PCA(X_test))

#Previous inputs for model training : X_train_pca, y_smo2_c
#Inputs for model validation : X_test_pca, y_test

y_pred_lat = LAT.predict(X_test_pca)
y_pred_ord = ORD.predict(X_test_pca)
print(y_pred_lat.shape)
print(y_pred_ord.shape)

(12653,)
(12653,)


In [ ]:
Counter(y_pred_lat)

Counter({0: 16405, 1: 18, 2: 16, 3: 44, 4: 67, 5: 138, 6: 21, 7: 2})

In [ ]:
Counter(y_pred_ord)

Counter({1.0: 12, 2.0: 2374, 3.0: 12896, 4.0: 1419, 5.0: 10})

In [ ]:
#LogisticAT model

#indexes of misclassified elements
compRes = [i for i in range(len(y_test)) if y_pred_lat[i] != y_test[i]]

#Number of elements inside validation set
print(len(compRes))

#Percentage of misclassified elements inside validation set
print("Percentage of misclassified (LogisticAT): ", (len(compRes) / len(y_test))*100)

6526
Percentage of misclassified (LogisticAT):  39.052121357189876


In [ ]:
#LogisticAT model

#Percentage of misclassified elements inside validation set, for each classes
ConfMat(y_pred_lat, y_test, 11)

Class 	Misclassified 	    Percentage
0	7878.0			100.0
1	17.0			100.0
2	28.0			93.33
3	63.0			92.65
4	80.0			83.33
5	378.0			70.79
6	294.0			75.77
7	719.0			86.94
8	1081.0			93.19
9	715.0			99.44
10	936.0			100.0

TOTAL	12189.0			96.33


In [ ]:
#OrdinalRidge model

#indexes of misclassified elements
compRes = [i for i in range(len(y_test)) if y_pred_ord[i] != y_test[i]]

#Percentage of misclassified elements inside validation set
print("Percentage of misclassified (OrdinalRidge): ", (len(compRes) / len(y_test))*100)

Percentage of misclassified (OrdinalRidge):  96.1274006164546


In [ ]:
#OrdinalRidge model

#Percentage of misclassified elements inside validation set, for each classes
ConfMat(y_pred_ord, y_test, 11)

Class 	Misclassified 	    Percentage
0	7878.0			100.0
1	17.0			100.0
2	29.0			96.67
3	63.0			92.65
4	80.0			83.33
5	305.0			57.12
6	281.0			72.42
7	771.0			93.23
8	1130.0			97.41
9	718.0			99.86
10	936.0			100.0

TOTAL	12208.0			96.48


##### Binary ratings

In [ ]:
#recall view of confusion matrix (among each true classes, how many correctly classified... and the remaining)
def conf_pour(matr):
  conf_pour = matr
  conf_pour = conf_pour.astype('float64')
  conf_pour[0][0] = 100*(round(matr[0][0] / (matr[0][0] + matr[0][1]), 4))
  conf_pour[0][1] = 100*(round(matr[0][1] / (matr[0][0] + matr[0][1]), 4))
  conf_pour[1][0] = 100*(round(matr[1][0] / (matr[1][0] + matr[1][1]), 4))
  conf_pour[1][1] = 100*(round(matr[1][1] / (matr[1][0] + matr[1][1]), 4))
  print("Confusion matrix (LogisticAT) :", conf_pour, sep = "\n")

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 4691, 1: 4867})

In [ ]:
y_pred_lat = LAT.predict(X_test)
y_pred_ord = ORD.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#LogisticAT model

#Confusion matrix
confusion_matrix_lat = confusion_matrix( y_pred_lat, y_test )
print(confusion_matrix_lat)

#recall view of confusion matrix
conf_pour(confusion_matrix_lat)

[[2375 2013]
 [2316 2854]]
Confusion matrix (LogisticAT) :
[[54.12 45.88]
 [44.8  55.2 ]]


In [ ]:
#LogisticAT model

print("Percentage of misclassified (LogisticAT): ", ((confusion_matrix_lat[0,1] + confusion_matrix_lat[1,0]) / len(y_test))*100)

Percentage of misclassified (LogisticAT):  45.29190207156309


In [ ]:
confusion_matrix_ord = confusion_matrix( y_pred_ord, y_test )
print(confusion_matrix_ord)

conf_pour(confusion_matrix_ord)

[[2370 2003]
 [2321 2864]]
Confusion matrix (LogisticAT) :
[[54.2  45.8 ]
 [44.76 55.24]]


In [ ]:
#OrdinalRidge model

print("Percentage of misclassified (OrdinalRidge): ", ((confusion_matrix_ord[0,1] + confusion_matrix_ord[1,0]) / len(y_test))*100)

Percentage of misclassified (OrdinalRidge):  45.23958987235823


# Classical regression

In [ ]:
#Custom round function to find the closest int from the provided float
def MyRound(x): 
  return math.ceil(x) if math.ceil(x) - x < x - math.floor(x) else math.floor(x)

In [ ]:
from sklearn.linear_model import LinearRegression
model_LinReg = LinearRegression()

In [ ]:
#Example of inputs used

#X_smo2_c (imblearn(X_train)), y_smo2_c (imblearn(y_train)) are the 5th test to balance classes
#X_train_pca (PCA(X_smo2_c)), X_test_pca (PCA(X_test))

#Inputs for model training : X_train_pca, y_smo2_c

#Model TRAINING
model_LinReg = LinearRegression()
model_LinReg.fit(X_train_pca, np.squeeze(y_smo2_c))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#Model VALIDATION
#Inputs for model validation : X_test_pca, y_test
y_lin_pred = model_LinReg.predict(X_test_pca)

In [ ]:
#Regression model prediction values are rounded 
y_lin_pred = list(map(MyRound, y_lin_pred))
Counter(y_lin_pred)

Counter({1: 12, 2: 2374, 3: 12896, 4: 1419, 5: 10})

In [ ]:
Counter(np.squeeze(y_test))

Counter({0: 10341,
         1: 25,
         2: 40,
         3: 91,
         4: 127,
         5: 698,
         6: 522,
         7: 1094,
         8: 1523,
         9: 968,
         10: 1282})

In [ ]:
#indexes of misclassified elements
compRes = [i for i in range(len(y_test)) if y_lin_pred[i] != y_test[i]]

#Percentage of misclassified elements inside validation set
print("Percentage of misclassified (LinearRegression): ", (len(compRes) / len(y_test))*100)

Percentage of misclassified (LinearRegression):  96.19852999288706


In [ ]:
#Percentage of misclassified elements inside validation set, for each classes
ConfMat(y_lin_pred, y_test, 11)

Class 	Misclassified 	    Percentage
0	7878.0			100.0
1	17.0			100.0
2	30.0			100.0
3	68.0			100.0
4	96.0			100.0
5	534.0			100.0
6	388.0			100.0
7	827.0			100.0
8	1160.0			100.0
9	719.0			100.0
10	936.0			100.0

TOTAL	12653.0			100.0


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modele_rf = RandomForestClassifier()

##### Unbalanced ratings

In [ ]:
#Example of inputs used

#X_smo2_c (imblearn(X_train)), y_smo2_c (imblearn(y_train)) are the 5th test to balance classes
#X_train_pca (PCA(X_smo2_c)), X_test_pca (PCA(X_test))

#Inputs for model training : X_train_pca, y_smo2_c

modele_rf.fit(X_train_pca, np.squeeze(y_smo2_c))	

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#Model validation
#Inputs for model validation : X_test_pca, y_test
y_predict_rf = modele_rf.predict(X_test_pca)
Counter(y_predict_rf)	

Counter({0: 2063,
         1: 65,
         2: 128,
         3: 315,
         4: 449,
         5: 1447,
         6: 1101,
         7: 2097,
         8: 1663,
         9: 1568,
         10: 1757})

In [ ]:
#indexes of misclassified elements
compRes = [i for i in range(len(y_test)) if y_predict_rf[i] != y_test[i]]

#Percentage of misclassified elements inside validation set
print("Percentage of misclassified (RandomForestClassifier): ", (len(compRes) / len(y_test))*100)

Percentage of misclassified (RandomForestClassifier):  64.93321741879396


In [ ]:
#Percentage of misclassified elements inside validation set, for each classes
ConfMat(y_predict_rf, y_test, 11) 

Class 	Misclassified 	    Percentage
0	6498.0			82.48
1	17.0			100.0
2	28.0			93.33
3	65.0			95.59
4	91.0			94.79
5	472.0			88.39
6	355.0			91.49
7	682.0			82.47
8	972.0			83.79
9	603.0			83.87
10	724.0			77.35

TOTAL	10507.0			83.04


##### Binary ratings

In [ ]:
modele_rf.fit(X_train, np.squeeze(y_train))	

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_predict_rf = modele_rf.predict(X_test)

In [ ]:
#confusion matrix
confusion_matrix_rf = confusion_matrix( y_predict_rf, y_test )
print(confusion_matrix_rf)

#recall view of confusion matrix
conf_pour(confusion_matrix_rf)

[[2435 2133]
 [2256 2734]]
Confusion matrix (LogisticAT) :
[[53.31 46.69]
 [45.21 54.79]]


In [ ]:
print("Percentage of misclassified (RandomForestClassifier): ", ((confusion_matrix_rf[0,1] + confusion_matrix_rf[1,0]) / len(y_test))*100)

Percentage of misclassified (RandomForestClassifier):  45.91964846202134


# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#no convergence with default solver ’lbfgs’ (even with max_iter = 10 000)
model_LogReg = LogisticRegression(max_iter = 10000, solver = "liblinear")

In [ ]:
#Model training
model_LogReg.fit(X_train, np.squeeze(y_train))	

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#Model validation
y_predict_LogReg = model_LogReg.predict(X_test)

In [ ]:
#confusion matrix
confusion_matrix_LogReg = confusion_matrix( y_predict_LogReg, y_test )
print(confusion_matrix_LogReg)

#recall view of confusion matrix
conf_pour(confusion_matrix_LogReg)

[[2065 1837]
 [2626 3030]]
Confusion matrix (LogisticAT) :
[[52.92 47.08]
 [46.43 53.57]]


In [ ]:
print("Percentage of misclassified (LogisticRegression): ", ((confusion_matrix_LogReg[0,1] + confusion_matrix_LogReg[1,0]) / len(y_test))*100)

Percentage of misclassified (LogisticRegression):  46.693869010253195
